In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

import librosa
import os
import seaborn as sns
from matplotlib.pyplot import specgram
from sklearn.model_selection import KFold
import librosa.display
import IPython as ipd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers

%matplotlib inline
%reload_ext tensorboard

In [26]:
csv_path="/content/drive/MyDrive/Datasets-Collab/Urban Sound Classification"
audio_path="/content/drive/MyDrive/train.zip (Unzipped Files)/Train/"
processed_path="/content/drive/MyDrive/Datasets-Collab/Urban Sound Classification/processed/"

In [27]:
df_train=pd.read_csv(csv_path+'/Train.csv')
df_test=pd.read_csv(csv_path+'/Test.csv')
submission=pd.read_csv(csv_path+'/sample_submission.csv')

In [28]:
from sklearn.preprocessing import LabelEncoder
Label_encoder=LabelEncoder()

df_train['Class']=Label_encoder.fit_transform(df_train['Class'])

In [93]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(file_name,label):
    fn=file_name
    features, labels = np.empty((0,193)), np.empty(0) # 193 => total features 
    mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
    ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    features = np.vstack([features,ext_features])
    labels = np.append(labels,label)
    return np.array(features, dtype=np.float32), np.array(labels, dtype = np.int8)

In [107]:
# temp=df_train.head(10)
# sound_path=[]
# for i,label in zip(temp['ID'],temp['Class']):
#   sound_path=audio_path+str(i)+'.wav'
#   features, labels = parse_audio_files(sound_path,label)
#   np.savez(processed_path+str(i), features=features, 
#              labels=labels)

In [108]:
# temp=df_train
# sound_path=[]
# for i,label in zip(temp['ID'],temp['Class']):
#   sound_path=audio_path+str(i)+'.wav'
#   features, labels = parse_audio_files(sound_path,label)
#   np.savez(processed_path+str(i), features=features, 
#              labels=labels)


In [145]:
split=int(0.7*df_train.shape[0])
x_train, y_train = [], []
x_test, y_test= [] , []
i=0
for files in df_train['ID']:
  files=str(files)+'.npz'
  data=np.load(processed_path+files)
  if (i<split):
    x_train.append(data["features"])
    y_train.append(data["labels"])
  else:
    x_test.append(data["features"])
    y_test.append(data["labels"])
  i=i+1


## Training the model

In [146]:
x_train=np.concatenate(x_train, axis = 0)
y_train = np.concatenate(y_train, axis = 0)
x_test=np.concatenate(x_test, axis = 0)
y_test = np.concatenate(y_test, axis = 0)


In [149]:
def get_network():
    input_shape = (193,)
    num_classes = 10
    keras.backend.clear_session()
    
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(256, activation="relu", input_shape=input_shape))
    model.add(keras.layers.Dense(128, activation="relu", input_shape=input_shape))
    model.add(keras.layers.Dense(64, activation="relu", input_shape=input_shape))
    model.add(keras.layers.Dense(num_classes, activation = "softmax"))
    model.compile(optimizer=keras.optimizers.Adam(1e-4), 
        loss=keras.losses.SparseCategoricalCrossentropy(), 
        metrics=["accuracy"])
    
    return model

In [152]:
accuracies = []
model = get_network()
model.fit(x_train, y_train, epochs = 100, batch_size = 24, validation_data=(x_test,y_test) ,verbose = 0)
l, a = model.evaluate(x_test, y_test, verbose = 0)
accuracies.append(a)
print("Loss: {0} | Accuracy: {1}".format(l, a))

Loss: 0.7615966200828552 | Accuracy: 0.9031268954277039
